In [ ]:
- vanishing and exploding gradients
-- batch norm
-- xavier initialization
-- Relu activation 

- regularisation
-- dropout
-- standard

- normalisation

- loss functions

## Vanishing and Exploding gradients
Two key problems that arise when training neural networks are vanishing and exploding gradients. These problems are when we have deep networks and are due to the fact that the first layers' gradients are calculated using the chain rule which requires us to multiply with the gradients of the final layer neurons. Consider the case when we are using a sigmoid activation function with the network shown below. As you can see each time we calculate the gradients for nodes closer to the input, we multiply by a $\sigma'(z_i)$ term. Given that the max value that this derivative of the sigmoid can take is 0.25 as shown in the graph below, our gradients will reduce by atleast a factor of 4 each time we add a layer. The gradient vanishes which makes learning very slow in the earlier layers. This is especially a problem if the neuron (z-value) is satuarated meaning it takes on a very high or very low value. This is because the derivative for very high and low sigmoid inputs is practically 0.

![](nngrad.png)

$J = \frac{1}{2}(h-y)^2$

$h = \sigma(z_3)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$z_3 = w_3a_2$

$a_2 = \sigma(z_2)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$z_2 = w_2a_1$

$a_1 = \sigma(z_1)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$z_1 = w_1x$

$\frac{\partial J}{\partial w_3} = (h-y)\ \sigma'(z_3)\ a_2$

$\frac{\partial J}{\partial w_2} = (h-y)\ \sigma'(z_3)\ w_3\ \sigma'(z_2)\ a_1$

$\frac{\partial J}{\partial w_1} = (h-y)\ \sigma'(z_3)\ w_3\ \sigma'(z_2)\ w_2\ \sigma'(z_1)\ x$

![](sigderiv.png)

In certain cases, the w terms can get very high and after being multiplied all the way through the network can lead to an extremely high gradient causing us to jump about all over the cost surface, never converging. This is called exploding gradients and occurs less frequently than vanishing gradient.

## Weights Initialization
Due to the problems explained above, we have to be careful in how we initialize our weights. If all of our weights have very high positive or negative values initially, then our neurons will satuarate and we will get very small gradients leading to slow learning or no learning at all. This is also another reason to normalize your inputs as you can imagine very high and low inputs will quickly lead to neuron satuaration.

We can smartly initialize our weights using Xavier Initialization. This means that each weight in our network is randomly samples from a Normal distribution with mean 0 and variance $\frac{1}{N_{avg}}$ where $N_{avg}$ is the average of the number of input neurons and output neurons to the layer the weights are being initialized for.

This usually isn't the first thing to add when improving performance as the other techniques described lead to larger gains but this can be good to give a small boost.
#### Implementation
We can implement this in pytorch by creating a new class inheriting from the defalt layer type we are trying to initialize but overriding the default reset_parameters function to initialize the weights differently.

In [ ]:
class Linear(nn.Linear):
    def reset_parameters(self):
        var = 2 / (self.in_features + self.out_features)
        self.weight.data.normal_(0, np.sqrt(var)) #takes in stdv as argument so we square-root variance to get stdv
        if self.bias is not None:
            self.bias.data.zero_()

class Conv2d(nn.Conv2d):
    def reset_parameters(self):
        var = 2 / ((self.in_channels + self.out_channels) * np.prod(self.kernel_size))
        self.weight.data.normal_(0, np.sqrt(var))
        if self.bias is not None:
            self.bias.data.zero_()

## Batch Normalization
Usually, we normalise our input features to help our network train faster. Batch-norm takes advantage of this trick by normalizing the output z-values at each layer with respect to the current batch. We then multiply by learnable parameter $\gamma$ and add learnable parameter $\beta$ to allow us to learn the best mean and variance for the z-values.
A more in-depth explanation can be found here: https://youtu.be/tNIpEZLv_eg

$z_{norm} = \frac{z-\mu}{\sigma^2}$

$\stackrel{\sim}{z} = \gamma z_{norm} + \beta$

This is one of the best tricks to give a big boost to performance especially in deeper architectures.

#### Implementation
We implement by defining batch norm operations which we apply after applying each layer except the output layer. The batch norm operation takes in the number of filters as the argument if it is a 2d batch-norm otherwise the number of neurons if it is a dense layer.

In [ ]:
class discriminator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1)
        self.dense1 = torch.nn.Linear(64*14*14, 256)
        self.dense2 = torch.nn.Linear(256, 1)

        self.bn1 = torch.nn.BatchNorm2d(64)
        self.bn2 = torch.nn.BatchNorm1d(256)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))).view(-1, 64*14*14)
        x = F.relu(self.bn2(self.dense1(x)))
        x = F.sigmoid(self.dense2(x))
        return x

## Activation functions
We can sometimes significantly improve performance by experimenting with different activation functions. Relu has been in the spotlight in recent years, often achieving the best performance but there are always new ones being researched. Take, for example, the Selu activation, which, according to the paper I read shattered some benchmarks and is easy to implement.

$selu(x) = \lambda \begin{cases}x & x>0 \\ \alpha e^x - \alpha & x<=0 \end{cases}$

#### Implementation
We simply define the new activation function and apply it on our forward pass

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__() #call parent class initializer
        self.h1 = torch.nn.Linear(30, 10) #input layer to size 10 hidden layer
        self.out = torch.nn.Linear(10, 1) #hidden layer to single output

    #define the forward propagation/prediction equation of our model
    def forward(self, x):
        x = self.h1(x) #linear combination
        x = self.selu(x) #activation
        x = self.out(x) #linear combination
        x = F.sigmoid(x) #activation
        return x
    
    def selu(self, x):
        scale = 1.0507009873554804934193349852946
        alpha = 1.6732632423543772848170429916717
        x[x>0] = scale*x
        x[x<=0] = scale*(alpha*x.exp() - alpha)
        return x